In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon

In [2]:
def predict(tick, given_treecomplexity, given_learningrate, given_subsample):
    #risk factors
    risk_factor_data=pd.read_excel(r'input/tick_risk_factor.xlsx')
    risk_factor_data['id']=risk_factor_data['id'].astype(str)
    risk_factor_data['country']=risk_factor_data['country'].astype(str)
    risk_factor_data['province']=risk_factor_data['province'].astype(str)
    risk_factor_data['county']=risk_factor_data['county'].astype(str)
    risk_factors=risk_factor_data.columns[4:]

    
    #tick data
    tick_data=pd.read_excel(r'input/tick_county_distribution.xlsx')
    tick_data=tick_data[['id', 'country', 'province', 'county', tick]]
    tick_data['id']=tick_data['id'].astype(str)
    tick_data['country']=tick_data['country'].astype(str)
    tick_data['province']=tick_data['province'].astype(str)
    tick_data['county']=tick_data['county'].astype(str)
    
    #combine and split
    complete_data=risk_factor_data.merge(tick_data, left_on=['id', 'country', 'province', 'county'], right_on=['id', 'country', 'province', 'county'], how='left')
    complete_data[tick]=complete_data[tick].fillna(-999)
    survey_data=complete_data[complete_data[tick]!=-999].reset_index(drop=True)
    # print(survey_data.shape[0])

    #add weight
    weight=pd.read_excel(r'input/weight.xlsx')
    weight['id']=weight['id'].astype(str)
    weight['country']=weight['country'].astype(str)
    weight['province']=weight['province'].astype(str)
    weight['county']=weight['county'].astype(str)
    weight=weight[['id', 'country', 'province', 'county', 'rescaled']]
    # print(weight.shape[0])

    survey_data=survey_data.merge(weight, left_on=['id', 'country', 'province', 'county'], right_on=['id', 'country', 'province', 'county'], how='left')
    survey_data[tick]=survey_data[tick].apply(lambda x: 1 if x > 0 else 0)
    
    #step 1, identify key risk factors
    rel_con=pd.DataFrame({'Key': risk_factors})   
    for i in range(10):
            train, test = train_test_split(survey_data, test_size=0.25,  random_state=i)
            while test[test[tick]==1].shape[0]<1 or train[train[tick]==1].shape[0]<1:
                train, test = train_test_split(survey_data, test_size=0.25)
            train_x = train[risk_factors]
            train_y = train[tick]
            train_weight =  train['rescaled']
            model = xgb.XGBClassifier(objective='binary:logistic', learning_rate=given_learningrate, max_depth=given_treecomplexity, subsample=given_subsample, n_estimators=3000,  random_state=i) # colsample_bytree=1
            model.fit(train_x, train_y, sample_weight=train_weight)
            importances = model.get_booster().get_score(importance_type='weight')
            importances_df=pd.DataFrame(list(importances.items()), columns=['Key', 'Value'])
            importances_df=importances_df.rename(columns={'Value': 'Value_%d'%i})
            rel_con=rel_con.merge(importances_df, left_on='Key', right_on='Key', how='left')
    rel_con['sum'] = rel_con.iloc[:, 1:].sum(axis=1)
    rel_con['rc']=rel_con['sum']/rel_con['sum'].sum()
    key_risk_df=rel_con[rel_con['rc']>=0.02].reset_index(drop=True)
    key_risk_factors=key_risk_df['Key'].values
    
    #step 2, predict use key risk factors
    rel_con=pd.DataFrame({'Key': key_risk_factors})
    preds=[]
    auc_value=[]
    tpr_value=[]
    tnr_value=[]
    high_risk=[]
    for i in range(100):
        train, test = train_test_split(survey_data, test_size=0.25,  random_state=i)
        while test[test[tick]==1].shape[0]<1 or train[train[tick]==1].shape[0]<1:
            train, test = train_test_split(survey_data, test_size=0.25)

        train_x = train[key_risk_factors]
        train_y = train[tick]
        train_weight =  train['rescaled']
        model = xgb.XGBClassifier(objective='binary:logistic', learning_rate=given_learningrate, max_depth=given_treecomplexity, subsample=given_subsample, n_estimators=3000,  random_state=i) # colsample_bytree=1
        model.fit(train_x, train_y, sample_weight=train_weight)
        importances = model.get_booster().get_score(importance_type='weight')
        importances_df=pd.DataFrame(list(importances.items()), columns=['Key', 'Value'])
        importances_df=importances_df.rename(columns={'Value': 'Value_%d'%i})
        rel_con=rel_con.merge(importances_df, left_on='Key', right_on='Key', how='left')
        
        test_x = test[key_risk_factors]
        test_y = test[tick]
        pred_test_y=model.predict_proba(test_x)[:,1]
        fpr, tpr, thresholds = roc_curve(test_y, pred_test_y)
        roc_auc = auc(fpr, tpr)
        auc_value.append(roc_auc)
        optimal_idx = np.argmax(tpr - fpr)  # 找到最大约登指数对应的索引
        optimal_threshold = thresholds[optimal_idx]  # 最佳阈值
        optimal_fpr = fpr[optimal_idx]  # 最佳阈值对应的FPR
        optimal_tpr = tpr[optimal_idx]  # 最佳阈值对应的TPR
        tpr_value.append(optimal_tpr)
        tnr_value.append(1-optimal_fpr)
        # optimal_threshold = thresholds[np.argmax(tpr - fpr)]          

        all_x=complete_data[key_risk_factors]
        pred_all_y=model.predict_proba(all_x)[:,1]
        pred_df=pd.DataFrame({'id':complete_data['id'], 'country': complete_data['country'], 'province': complete_data['province'], 'county':complete_data['county'], 'round': [i]*complete_data.shape[0], 'status':complete_data[tick],'pred':pred_all_y,'sensitivity':[optimal_tpr]*complete_data.shape[0],'specificity': [(1-optimal_fpr)]*complete_data.shape[0]})
        preds.append(pred_df)

        high_risk_data=pred_df[pred_df['pred']>=optimal_threshold].reset_index(drop=True)
        high_risk_data['threshold']=[optimal_threshold]*high_risk_data.shape[0]
        high_risk.append(high_risk_data)

    #mean auc of testing set
    auc_mean=np.mean(auc_value)
    auc_ubd=np.percentile(auc_value, 97.5)
    auc_lbd=np.percentile(auc_value, 2.5)
    auc_df=pd.DataFrame({'index': ['mean', 'lbd', 'ubd'], 'value': [auc_mean, auc_lbd, auc_ubd]})
    auc_df.to_excel(r'output/auc_%s.xlsx'%tick, index=False)

    tpr_mean=np.mean(tpr_value)
    tpr_ubd=np.percentile(tpr_value, 97.5)
    tpr_lbd=np.percentile(tpr_value, 2.5)
    tpr_df=pd.DataFrame({'index': ['mean', 'lbd', 'ubd'], 'value': [tpr_mean, tpr_lbd, tpr_ubd]})
    tpr_df.to_excel(r'output/tpr_%s.xlsx'%tick, index=False)

    tnr_mean=np.mean(tnr_value)
    tnr_ubd=np.percentile(tnr_value, 97.5)
    tnr_lbd=np.percentile(tnr_value, 2.5)
    tnr_df=pd.DataFrame({'index': ['mean', 'lbd', 'ubd'], 'value': [tnr_mean, tnr_lbd, tnr_ubd]})
    tnr_df.to_excel(r'output/tnr_%s.xlsx'%tick, index=False)


    #relative contribution
    rel_con['row_sum']=rel_con.iloc[:, 1:].sum(axis=1)
    rel_con['std_sum']=rel_con['row_sum']/rel_con['row_sum'].sum()
    rel_con.to_excel(r'output/rc_%s.xlsx'%tick, index=False)
    
    #high_risk_counties
    high_risk_df=pd.concat(high_risk, ignore_index=True)
    high_risk_df.to_excel(r'output/hrc_%s.xlsx'%tick, index=False)

    #calcualte mean, std and 95% CI
    pred_df=pd.concat(preds, ignore_index=True)
    grouped = pred_df.groupby(['id', 'country', 'province', 'county', 'status'])['pred'].mean()
    grouped = grouped.reset_index()
    grouped.to_excel(r'output/predict_prob_%s.xlsx'%tick, index=False)

In [ ]:
ticks=['Dermacentor marginatus', 'Dermacentor nuttalli', 'Dermacentor silvarum',  'Hyalomma asiaticum']
complexitys=[7, 7, 5, 5]
learningrates=[0.001, 0.001, 0.001, 0.01]
subsamples=[0.7, 0.7, 0.7, 0.8]
for tick, complexity, learningrate, subsample in zip(ticks, complexitys, learningrates, subsamples):
    print(tick, complexity, learningrate, subsample)
    predict(tick, complexity, learningrate, subsample)

Dermacentor marginatus 7 0.001 0.7


In [2]:
#AUC
df=pd.DataFrame({'index': ['mean', 'lbd', 'ubd']})
ticks=['Dermacentor marginatus', 'Dermacentor nuttalli', 'Dermacentor silvarum',  'Hyalomma asiaticum']
for i in range(len(ticks)):
    tmp=pd.read_excel(r'output/auc_%s.xlsx'%ticks[i])
    df[ticks[i]]=tmp['value']
dfn=df.T
dfn.to_excel(r'output/AUC_tick.xlsx')

In [3]:
df=pd.DataFrame({'index': ['mean', 'lbd', 'ubd']})
ticks=['Dermacentor marginatus', 'Dermacentor nuttalli', 'Dermacentor silvarum',  'Hyalomma asiaticum']
for i in range(len(ticks)):
    tmp=pd.read_excel(r'output/tpr_%s.xlsx'%ticks[i])
    df[ticks[i]]=tmp['value']
dfn=df.T
dfn.to_excel(r'output/TPR_tick.xlsx')

In [4]:
df=pd.DataFrame({'index': ['mean', 'lbd', 'ubd']})
ticks=['Dermacentor marginatus', 'Dermacentor nuttalli', 'Dermacentor silvarum',  'Hyalomma asiaticum']
for i in range(len(ticks)):
    tmp=pd.read_excel(r'output/tnr_%s.xlsx'%ticks[i])
    df[ticks[i]]=tmp['value']
dfn=df.T
dfn.to_excel(r'output/TNR_tick.xlsx')

In [5]:
#relative contribution
risk_factor=pd.read_excel(r'output/risk_factor/tick.xlsx')
df=pd.DataFrame({'factor': risk_factor.columns[4:]})
ticks=['Dermacentor marginatus', 'Dermacentor nuttalli', 'Dermacentor silvarum',  'Hyalomma asiaticum']
for i in range(len(ticks)):
    tmp=pd.read_excel(r'output/rc_%s.xlsx'%ticks[i])
    tmp=tmp[['Key', 'std_sum']]
    tmp=tmp.rename(columns={'Key': 'factor', 'std_sum': ticks[i]})
    df=df.merge(tmp, left_on='factor', right_on='factor', how='left')
df.to_excel(r'output/RC_tick.xlsx')

In [6]:
ticks=['Dermacentor marginatus', 'Dermacentor nuttalli', 'Dermacentor silvarum',  'Hyalomma asiaticum']
for i in range(len(ticks)):
    #high_risk_county, scenario 1
    county_list=pd.read_excel(r'input/tick_risk_factor.xlsx')
    county_list=county_list[['id', 'country', 'province', 'county']]
    county_list['id']=county_list['id'].astype(str)
    county_list['country']=county_list['country'].astype(str)
    county_list['province']=county_list['province'].astype(str)
    county_list['county']=county_list['county'].astype(str)

    tick_data=pd.read_excel(r'output/hrc_%s.xlsx'%ticks[i])
    tick_data['id']=tick_data['id'].astype(str)
    tick_data['country']=tick_data['country'].astype(str)
    tick_data['province']=tick_data['province'].astype(str)
    tick_data['county']=tick_data['county'].astype(str)
    for j in range(100):
        tmp=tick_data[tick_data['round']==j].reset_index(drop=True)
        new_column='flag_%d'%j
        tmp[new_column]=[1]*tmp.shape[0]
        tmp=tmp[['id','country','province','county', new_column]]
        county_list=county_list.merge(tmp, left_on=['id','country','province','county'], right_on=['id','country','province','county'], how='left')
    county_list.fillna(0, inplace=True)
    county_list['status']=county_list.iloc[:, 4:].sum(axis=1)
    df=county_list[county_list['status']>0].reset_index(drop=True)
    df=df[['id','country','province','county', 'status']]
    # df.rename(columns={'status': ticks[i]}, inplace=True)
    df.to_excel(r'output/hrc_clean_%s.xlsx'%ticks[i], index=False)
    print(ticks[i], df.shape[0])

Dermacentor marginatus 809
Dermacentor nuttalli 1069
Dermacentor silvarum 1154
Hyalomma asiaticum 890


In [7]:
obv_number=[]
obv_area=[]
obv_pop=[]
pred_number=[]
pred_area=[]
pred_pop=[]
col_tick=[]
for tick in ticks:
    pred=pd.read_excel(r'output/hrc_clean_%s.xlsx'%tick)
    pred['id']=pred['id'].astype(str)
    pred['country']=pred['country'].astype(str)
    pred['province']=pred['province'].astype(str)
    pred['county']=pred['county'].astype(str)

    obv=pd.read_excel(r'input/tick_county_distribution.xlsx')
    obv=obv[['id', 'country', 'province', 'county', tick]]
    obv=obv[obv[tick]>=1].reset_index(drop=True)
    obv['id']=obv['id'].astype(str)
    obv['country']=obv['country'].astype(str)
    obv['province']=obv['province'].astype(str)
    obv['county']=obv['county'].astype(str)
    
    county_information=pd.read_excel(r'input/population.xlsx')
    county_information['population']=county_information['area (km²)']*county_information['population density (/km²)']
    county_information['population']=county_information['population'].astype('int')
    county_information['id']=county_information['id'].astype(str)
    county_information['country']=county_information['country'].astype(str)
    county_information['province']=county_information['province'].astype(str)
    county_information['county']=county_information['county'].astype(str)

    number=0
    area=0
    population=0
    for j in range(pred.shape[0]):
        id=pred['id'][j]
        country=pred['country'][j]
        province=pred['province'][j]
        county=pred['county'][j]
        tmp=county_information[(county_information['id']==id) & (county_information['country']==country) & (county_information['province']==province) & (county_information['county']==county)].reset_index(drop=True)
        if tmp.shape[0]!=1:
            print('pred', tmp)
        else:
            number+=1
            area+=(tmp['area (km²)'][0])
            population+=(tmp['population'][0])
    pred_number.append(number)
    pred_area.append(area)
    pred_pop.append(population)

    number=0
    area=0
    population=0
    for j in range(obv.shape[0]):
        id=obv['id'][j]
        country=obv['country'][j]
        province=obv['province'][j]
        county=obv['county'][j]
        tmp=county_information[(county_information['id']==id) & (county_information['country']==country) & (county_information['province']==province) & (county_information['county']==county)].reset_index(drop=True)
        if tmp.shape[0]!=1:
            print('pred', tmp)
        else:
            number+=1
            area+=(tmp['area (km²)'][0])
            population+=(tmp['population'][0])
    obv_number.append(number)
    obv_area.append(area)
    obv_pop.append(population)   

    col_tick.append(tick)

col_1=[]
col_2=[]
col_3=[]
for i in range(len(col_tick)):
    obv1=obv_number[i]
    pred1=pred_number[i]
    ratio1=100*(pred1-obv1)/obv1
    print(col_tick[i], obv1, pred1, ratio1)
    col_1.append("%d/%d (%.1f)"%(pred1, obv1, ratio1))

    obv2=obv_area[i]/1e6
    pred2=pred_area[i]/1e6
    ratio2=100*(pred2-obv2)/obv2
    col_2.append("%.2f/%.2f (%.1f)"%(pred2, obv2, ratio2))

    obv3=obv_pop[i]/1e6
    pred3=pred_pop[i]/1e6
    ratio3=100*(pred3-obv3)/obv3
    col_3.append("%2.f/%.2f (%.1f)"%(pred3, obv3, ratio3))

dfn=pd.DataFrame({'tick': col_tick, 'county': col_1, 'area': col_2, 'population': col_3})
dfn.to_excel(r'output/PC_tick.xlsx', index=False) 

Dermacentor marginatus 81 809 898.7654320987655
Dermacentor nuttalli 120 1069 790.8333333333334
Dermacentor silvarum 66 1154 1648.4848484848485
Hyalomma asiaticum 122 890 629.5081967213115
